**Set Environment**

In [1]:
import sys
sys.path.append('../')
from config_sing import *

You are on Duke Server: Singularity: Proj CombEffect
BASE DIRECTORY:     /mount/work
PATH OF SOURCE:     /mount/work/source
PATH OF EXECUTABLE: /mount/work/exe
PATH OF ANNOTATION: /mount/work/annotation
PATH OF PROJECT:    /mount/project
PATH OF RESULTS:    /mount/work/out/proj_combeffect


In [2]:
import sqlite3
import itertools as it

In [3]:
fdiry = FD_ANN
os.listdir(fdiry)

['motif_cluster_jvierstra', 'genome', '.ipynb_checkpoints', 'log']

In [4]:
#fdiry = os.path.join(FD_ANN, "motif_cluster_jvierstra", "hg38_archetype_motifs_v1")
#os.listdir(fdiry)

fglob = os.path.join(FD_ANN, "motif_cluster_jvierstra", "hg38_archetype_motifs_v1", "chr17*")
glob.glob(fglob)

['/mount/work/annotation/motif_cluster_jvierstra/hg38_archetype_motifs_v1/chr17_KI270730v1_random.bed.gz',
 '/mount/work/annotation/motif_cluster_jvierstra/hg38_archetype_motifs_v1/chr17_KI270729v1_random.bed.gz',
 '/mount/work/annotation/motif_cluster_jvierstra/hg38_archetype_motifs_v1/chr17',
 '/mount/work/annotation/motif_cluster_jvierstra/hg38_archetype_motifs_v1/chr17_GL000205v2_random.bed.gz',
 '/mount/work/annotation/motif_cluster_jvierstra/hg38_archetype_motifs_v1/chr17.bed.gz',
 '/mount/work/annotation/motif_cluster_jvierstra/hg38_archetype_motifs_v1/chr17_rm_mouse.bed.gz',
 '/mount/work/annotation/motif_cluster_jvierstra/hg38_archetype_motifs_v1/chr17_rm_mouse_merge.bed.gz']

In [5]:
fdiry = os.path.join(FD_ANN, "motif_cluster_jvierstra", "hg38_archetype_motifs_v1")
fname = "chr17_rm_mouse_merge.bed.gz"
fpath = os.path.join(fdiry, fname)

print(fpath)

/mount/work/annotation/motif_cluster_jvierstra/hg38_archetype_motifs_v1/chr17_rm_mouse_merge.bed.gz


In [6]:
%%script env fpath="$fpath" bash
zcat ${fpath} | head

chr17	60004	60022	ZNF140	5.6897
chr17	60004	60022	ZNF667	8.024
chr17	60006	60015	Ebox/CAGCTG	7.9275
chr17	60011	60031	GC-tract	12.122
chr17	60012	60025	PRDM4	1.3083
chr17	60017	60028	NR/19	9.668
chr17	60019	60035	HEN1	5.5854
chr17	60023	60042	ZNF680	6.3901
chr17	60027	60037	SMARCA1	7.5566
chr17	60027	60040	LEF1	7.1402


In [8]:
def prep_line(lst):
    key = "_".join(lst[0:(len(lst)-1)])
    return [key] + lst

In [9]:
fdiry = os.path.join(FD_ANN, "motif_cluster_jvierstra", "hg38_archetype_motifs_v1")
fname = "chr17_rm_mouse_merge.bed.gz"
fpath = os.path.join(fdiry, fname)

n_lines = 10
with gzip.open(fpath, "rb") as file:
    lines = it.islice(file, n_lines)
    for line in lines:
        lst = line.decode('ASCII').strip().split('\t')  
        print(lst)
        print(prep_line(lst))

['chr17', '60004', '60022', 'ZNF140', '5.6897']
['chr17_60004_60022_ZNF140', 'chr17', '60004', '60022', 'ZNF140', '5.6897']
['chr17', '60004', '60022', 'ZNF667', '8.024']
['chr17_60004_60022_ZNF667', 'chr17', '60004', '60022', 'ZNF667', '8.024']
['chr17', '60006', '60015', 'Ebox/CAGCTG', '7.9275']
['chr17_60006_60015_Ebox/CAGCTG', 'chr17', '60006', '60015', 'Ebox/CAGCTG', '7.9275']
['chr17', '60011', '60031', 'GC-tract', '12.122']
['chr17_60011_60031_GC-tract', 'chr17', '60011', '60031', 'GC-tract', '12.122']
['chr17', '60012', '60025', 'PRDM4', '1.3083']
['chr17_60012_60025_PRDM4', 'chr17', '60012', '60025', 'PRDM4', '1.3083']
['chr17', '60017', '60028', 'NR/19', '9.668']
['chr17_60017_60028_NR/19', 'chr17', '60017', '60028', 'NR/19', '9.668']
['chr17', '60019', '60035', 'HEN1', '5.5854']
['chr17_60019_60035_HEN1', 'chr17', '60019', '60035', 'HEN1', '5.5854']
['chr17', '60023', '60042', 'ZNF680', '6.3901']
['chr17_60023_60042_ZNF680', 'chr17', '60023', '60042', 'ZNF680', '6.3901']
['c

In [10]:
query_reset = ("DROP TABLE IF EXISTS Motif")

query_table = ("""CREATE TABLE IF NOT EXISTS Motif(
    binding TEXT PRIMARY KEY, 
    chrom   TEXT,
    start   INTEGER,
    end     INTEGER,
    motif   TEXT,
    score   REAL
);""")

query_insert = ("INSERT OR IGNORE INTO Motif (binding,chrom,start,end,motif,score) VALUES (?,?,?,?,?,?)")

In [17]:
fdiry = os.path.join(FD_RES, 'database')
fname = "fragment.db"
fpath_db = os.path.join(fdiry, fname)

fdiry = os.path.join(FD_ANN, "motif_cluster_jvierstra", "hg38_archetype_motifs_v1")
fname = "chr17_rm_mouse_merge.bed.gz"
fpath_gz = os.path.join(fdiry, fname)

with sqlite3.connect(fpath_db) as conn, gzip.open(fpath_gz, "rb") as file:
    ### reset
    cursor = conn.cursor()
    query  = query_reset
    cursor.execute(query)
    
    ### create table
    query  = query_table
    cursor.execute(query)
    
    ### insert values
    query  = query_insert
    lines = it.islice(file, n_lines)
    for line in lines:
        lst = line.decode('ASCII').strip().split('\t')  
        lst = prep_line(lst)
        cursor.execute(query, lst)
    
    ### show that the table is created
    cursor.execute("SELECT * FROM Motif")
    for row in cursor.fetchall():
        print(row)

('chr17_60004_60022_ZNF140', 'chr17', 60004, 60022, 'ZNF140', 5.6897)
('chr17_60004_60022_ZNF667', 'chr17', 60004, 60022, 'ZNF667', 8.024)
('chr17_60006_60015_Ebox/CAGCTG', 'chr17', 60006, 60015, 'Ebox/CAGCTG', 7.9275)
('chr17_60011_60031_GC-tract', 'chr17', 60011, 60031, 'GC-tract', 12.122)
('chr17_60012_60025_PRDM4', 'chr17', 60012, 60025, 'PRDM4', 1.3083)
('chr17_60017_60028_NR/19', 'chr17', 60017, 60028, 'NR/19', 9.668)
('chr17_60019_60035_HEN1', 'chr17', 60019, 60035, 'HEN1', 5.5854)
('chr17_60023_60042_ZNF680', 'chr17', 60023, 60042, 'ZNF680', 6.3901)
('chr17_60027_60037_SMARCA1', 'chr17', 60027, 60037, 'SMARCA1', 7.5566)
('chr17_60027_60040_LEF1', 'chr17', 60027, 60040, 'LEF1', 7.1402)


## Insert the whole file

In [19]:
%%time

fdiry = os.path.join(FD_RES, 'database')
fname = "fragment.db"
fpath_db = os.path.join(fdiry, fname)

fdiry = os.path.join(FD_ANN, "motif_cluster_jvierstra", "hg38_archetype_motifs_v1")
fname = "chr17_rm_mouse_merge.bed.gz"
fpath_gz = os.path.join(fdiry, fname)

with sqlite3.connect(fpath_db) as conn, gzip.open(fpath_gz, "rb") as file:
    ### init
    cursor = conn.cursor()
    
    ### create table
    query  = query_table
    cursor.execute(query)
    
    ### insert values
    query = query_insert
    lines = file #it.islice(file, n_lines)
    for line in lines:
        lst = line.decode('ASCII').strip().split('\t')  
        lst = prep_line(lst)
        cursor.execute(query, lst)

CPU times: user 1min 55s, sys: 4.15 s, total: 1min 59s
Wall time: 2min 10s


**Check**

In [20]:
fdiry = os.path.join(FD_ANN, "motif_cluster_jvierstra", "hg38_archetype_motifs_v1")
fname = "chr17_rm_mouse_merge.bed.gz"
fpath = os.path.join(fdiry, fname)

print(fpath)

/mount/work/annotation/motif_cluster_jvierstra/hg38_archetype_motifs_v1/chr17_rm_mouse_merge.bed.gz


In [21]:
%%script env fpath="$fpath" bash
zcat ${fpath} | wc -l

14308212


In [3]:
%%time

fdiry = os.path.join(FD_RES, 'database')
fname = "fragment.db"
fpath_db = os.path.join(fdiry, fname)

with sqlite3.connect(fpath_db) as conn:
    cursor = conn.cursor()
    query = "select count(*) from Motif"
    cursor.execute(query)
    print(cursor.fetchall())

[(14308212,)]
CPU times: user 172 ms, sys: 590 ms, total: 762 ms
Wall time: 2.95 s


In [22]:
%%time

with sqlite3.connect(fpath_db) as conn:
    cursor = conn.cursor()
    query = "select count(*) from Motif"
    cursor.execute(query)
    print(cursor.fetchall())

[(14308212,)]
CPU times: user 146 ms, sys: 637 ms, total: 783 ms
Wall time: 16.5 s


**Indexing**

In [7]:
query_index_start = """CREATE INDEX idx_motif_start ON Motif (start)"""
query_index_end   = """CREATE INDEX idx_motif_end   ON Motif (end)"""
query_index_loc   = """CREATE INDEX idx_motif_loc   ON Motif (start, end)"""

In [5]:
%%time
fdiry = os.path.join(FD_RES, 'database')
fname = "fragment.db"
fpath_db = os.path.join(fdiry, fname)

with sqlite3.connect(fpath_db) as conn:
    cursor = conn.cursor()
    query = query_index_start
    cursor.execute(query)

CPU times: user 6.19 s, sys: 1.94 s, total: 8.13 s
Wall time: 8.24 s


In [6]:
%%time
fdiry = os.path.join(FD_RES, 'database')
fname = "fragment.db"
fpath_db = os.path.join(fdiry, fname)

with sqlite3.connect(fpath_db) as conn:
    cursor = conn.cursor()
    query = query_index_end
    cursor.execute(query)

CPU times: user 6.38 s, sys: 1.84 s, total: 8.22 s
Wall time: 8.36 s


In [9]:
%%time
fdiry = os.path.join(FD_RES, 'database')
fname = "fragment.db"
fpath_db = os.path.join(fdiry, fname)

with sqlite3.connect(fpath_db) as conn:
    cursor = conn.cursor()
    query = query_index_loc
    cursor.execute(query)

CPU times: user 7.1 s, sys: 2.29 s, total: 9.39 s
Wall time: 9.56 s


In [11]:
query_drop_index_start = """DROP INDEX idx_motif_start;"""
query_drop_index_end = """DROP INDEX idx_motif_end;"""

In [12]:
fdiry = os.path.join(FD_RES, 'database')
fname = "fragment.db"
fpath_db = os.path.join(fdiry, fname)

with sqlite3.connect(fpath_db) as conn:
    cursor = conn.cursor()
    query = query_drop_index_start
    cursor.execute(query)
    cursor = conn.cursor()
    query = query_drop_index_end
    cursor.execute(query)